 #  A Whale off the Port(folio)
 ---

 In this assignment, you'll get to use what you've learned this week to evaluate the performance among various algorithmic, hedge, and mutual fund portfolios and compare them against the S&P 500 Index.

In [32]:
# Initial imports
import pandas as pd
import numpy as np
import datetime as dt
from pathlib import Path

%matplotlib inline

In [33]:
# added as temp fix to bypass false positive errors with warning "A value is trying to be set on a copy of a slice from a DataFrame"
pd.options.mode.chained_assignment = None  # default='warn'

# Data Cleaning

In this section, you will need to read the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files:

* `whale_returns.csv`: Contains returns of some famous "whale" investors' portfolios.

* `algo_returns.csv`: Contains returns from the in-house trading algorithms from Harold's company.

* `sp500_history.csv`: Contains historical closing prices of the S&P 500 Index.

In [34]:

# # functions to fix data types
# def fix_date(df,date_column):
#     df[date_column] = df[date_column].astype('datetime64')
#     return df

# def fix_currency(df,currency_column_index):
#     new_df = df.copy()
#     for i in range(len(new_df)-1):
#         val = new_df.iloc[:, currency_column_index][i] 
#         new_val_str = val.replace('$','')
#         new_df.iloc[:, currency_column_index][i] = float(new_val_str)
#     return new_df

## Whale Returns

Read the Whale Portfolio daily returns and clean the data

In [35]:
# Reading whale returns
whale_csv = Path("Resources/whale_returns.csv")
whale = pd.read_csv(whale_csv)
whale # verify DataFrame head, tail, total rows and columns

,Date,SOROS FUND MANAGEMENT LLC,PAULSON & CO.INC.,TIGER GLOBAL MANAGEMENT LLC,BERKSHIRE HATHAWAY INC
0,2015-03-02,NaN,NaN,NaN,NaN
1,2015-03-03,-0.001266,-0.004981,-0.000496,-0.006569
2,2015-03-04,0.002230,0.003241,-0.002534,0.004213
3,2015-03-05,0.004016,0.004076,0.002355,0.006726
4,2015-03-06,-0.007905,-0.003574,-0.008481,-0.013098
...,...,...,...,...,...
1055,2019-04-25,-0.000285,-0.001291,-0.005153,0.004848
1056,2019-04-26,0.008149,0.009162,0.012355,0.010434
1057,2019-04-29,0.001254,0.002719,0.006251,0.005223
1058,2019-04-30,-0.001295,-0.002211,-0.000259,-0.003702


In [36]:
# Count nulls
whale.isna().sum()

Date                           0
SOROS FUND MANAGEMENT LLC      1
PAULSON & CO.INC.              1
TIGER GLOBAL MANAGEMENT LLC    1
BERKSHIRE HATHAWAY INC         1
dtype: int64

In [37]:
# Drop nulls
whale = whale.dropna()

In [38]:
whale.isna().sum() # for testing to verify nulls dropped

Date                           0
SOROS FUND MANAGEMENT LLC      0
PAULSON & CO.INC.              0
TIGER GLOBAL MANAGEMENT LLC    0
BERKSHIRE HATHAWAY INC         0
dtype: int64

In [39]:
# whale # for testing to view null-dropped head, tail, total rows and columns
whale.dtypes

Date                            object
SOROS FUND MANAGEMENT LLC      float64
PAULSON & CO.INC.              float64
TIGER GLOBAL MANAGEMENT LLC    float64
BERKSHIRE HATHAWAY INC         float64
dtype: object

## Algorithmic Daily Returns

Read the algorithmic daily returns and clean the data

In [40]:
# Reading algorithmic returns
algo_csv = Path("Resources/algo_returns.csv")
algo = pd.read_csv(algo_csv)
algo # verify DataFrame head, tail, total rows and columns

,Date,Algo 1,Algo 2
0,2014-05-28,0.001745,NaN
1,2014-05-29,0.003978,NaN
2,2014-05-30,0.004464,NaN
3,2014-06-02,0.005692,NaN
4,2014-06-03,0.005292,NaN
...,...,...,...
1236,2019-04-25,0.000682,-0.007247
1237,2019-04-26,0.002981,0.005256
1238,2019-04-29,0.005208,0.002829
1239,2019-04-30,-0.002944,-0.001570


In [41]:
# Count nulls
algo.isna().sum()

Date      0
Algo 1    0
Algo 2    6
dtype: int64

In [42]:
# Drop nulls
algo = algo.dropna()
algo.isna().sum() # verify nulls dropped

Date      0
Algo 1    0
Algo 2    0
dtype: int64

In [43]:
algo # view head, tail, and number of rows, columsn of cleaned data

,Date,Algo 1,Algo 2
6,2014-06-05,0.004062,0.013285
7,2014-06-06,0.001857,0.008284
8,2014-06-09,-0.005012,0.005668
9,2014-06-10,0.004406,-0.000735
10,2014-06-11,0.004760,-0.003761
...,...,...,...
1236,2019-04-25,0.000682,-0.007247
1237,2019-04-26,0.002981,0.005256
1238,2019-04-29,0.005208,0.002829
1239,2019-04-30,-0.002944,-0.001570


## S&P 500 Returns

Read the S&P 500 historic closing prices and create a new daily returns DataFrame from the data. 

In [44]:
# Reading S&P 500 Closing Prices
sp500_history_csv = Path("Resources/sp500_history.csv")
sp500 = pd.read_csv(sp500_history_csv)
sp500

,Date,Close
0,23-Apr-19,$2933.68
1,22-Apr-19,$2907.97
2,18-Apr-19,$2905.03
3,17-Apr-19,$2900.45
4,16-Apr-19,$2907.06
...,...,...
1644,5-Oct-12,$1460.93
1645,4-Oct-12,$1461.40
1646,3-Oct-12,$1450.99
1647,2-Oct-12,$1445.75


In [45]:
# Check Data Types
sp500.dtypes

Date     object
Close    object
dtype: object

In [46]:
# Fix Data Types (date)
def fix_date(df,date_column):
    df[date_column] = df[date_column].astype('datetime64')
    return df

def fix_currency(df,column_index):
    for i in range(len(df)):
        val = df.iloc[:, column_index][i]
        new_val = val.replace('$','')
        df.iloc[:, column_index][i] = new_val
        i += 1
    return df

def fix_datatype(df,column,datatype):
    df[column] = df[column].astype(datatype)
    return df

date = 'Date'
fix_date(sp500, date)

currency_column_index = 1
fix_currency(sp500,currency_column_index)
fix_datatype(sp500,'Close','float64')

,Date,Close
0,2019-04-23,2933.68
1,2019-04-22,2907.97
2,2019-04-18,2905.03
3,2019-04-17,2900.45
4,2019-04-16,2907.06
...,...,...
1644,2012-10-05,1460.93
1645,2012-10-04,1461.40
1646,2012-10-03,1450.99
1647,2012-10-02,1445.75


In [47]:
sp500.dtypes # check that correction was successful

Date     datetime64[ns]
Close           float64
dtype: object

In [48]:
# sort by date
sp500.sort_values('Date', ascending=True, inplace=True)
sp500

,Date,Close
1648,2012-10-01,1444.49
1647,2012-10-02,1445.75
1646,2012-10-03,1450.99
1645,2012-10-04,1461.40
1644,2012-10-05,1460.93
...,...,...
4,2019-04-16,2907.06
3,2019-04-17,2900.45
2,2019-04-18,2905.03
1,2019-04-22,2907.97


In [49]:
# Calculate Daily Returns
daily_returns = []
for i in range(len(sp500)-2):
    today = sp500['Close'][i]
    tomorrow =  sp500['Close'][i+1]
    daily_change = tomorrow - today
    daily_return = daily_change / today
    daily_returns.append(daily_return)
    i+=1
print(f"daily_returns[0] = {daily_returns[0]} (first entry)")
print(f"daily_returns[{len(daily_returns)}] = {daily_returns[-1]} (last entry)")

daily_returns[0] = -0.008763737012898488 (first entry)
daily_returns[1647] = -0.003611327438507508 (last entry)


In [50]:
# Calculate Daily Returns
returns_df = sp500['Close'].pct_change()
returns_df

1648         NaN
1647    0.000872
1646    0.003624
1645    0.007174
1644   -0.000322
          ...   
4       0.000509
3      -0.002274
2       0.001579
1       0.001012
0       0.008841
Name: Close, Length: 1649, dtype: float64

In [84]:
# Drop nulls
returns_df.dropna()
returns_df.isna().sum()
returns_df.isna()

1648     True
1647    False
1646    False
1645    False
1644    False
        ...  
4       False
3       False
2       False
1       False
0       False
Name: Close, Length: 1649, dtype: bool

In [88]:
# Rename `Close` Column to be specific to this portfolio.
returns_df.rename_axis("Daily Return", axis=1, inplace=True)

ValueError: No axis named 1 for object type Series

## Combine Whale, Algorithmic, and S&P 500 Returns

In [53]:
# Join Whale Returns, Algorithmic Returns, and the S&P 500 Returns into a single DataFrame with columns for each portfolio's returns.


---

# Conduct Quantitative Analysis

In this section, you will calculate and visualize performance and risk metrics for the portfolios.

## Performance Anlysis

#### Calculate and Plot the daily returns.

In [54]:
# Plot daily returns of all portfolios


#### Calculate and Plot cumulative returns.

In [55]:
# Calculate cumulative returns of all portfolios

# Plot cumulative returns


---

## Risk Analysis

Determine the _risk_ of each portfolio:

1. Create a box plot for each portfolio. 
2. Calculate the standard deviation for all portfolios
4. Determine which portfolios are riskier than the S&P 500
5. Calculate the Annualized Standard Deviation

### Create a box plot for each portfolio


In [56]:
# Box plot to visually show risk


### Calculate Standard Deviations

In [57]:
# Calculate the daily standard deviations of all portfolios


### Determine which portfolios are riskier than the S&P 500

In [58]:
# Calculate  the daily standard deviation of S&P 500

# Determine which portfolios are riskier than the S&P 500


### Calculate the Annualized Standard Deviation

In [59]:
# Calculate the annualized standard deviation (252 trading days)


---

## Rolling Statistics

Risk changes over time. Analyze the rolling statistics for Risk and Beta. 

1. Calculate and plot the rolling standard deviation for all portfolios using a 21-day window
2. Calculate the correlation between each stock to determine which portfolios may mimick the S&P 500
3. Choose one portfolio, then calculate and plot the 60-day rolling beta between it and the S&P 500

### Calculate and plot rolling `std` for all portfolios with 21-day window

In [60]:
# Calculate the rolling standard deviation for all portfolios using a 21-day window

# Plot the rolling standard deviation


### Calculate and plot the correlation

In [61]:
# Calculate the correlation

# Display de correlation matrix


### Calculate and Plot Beta for a chosen portfolio and the S&P 500

In [62]:
# Calculate covariance of a single portfolio

# Calculate variance of S&P 500

# Computing beta

# Plot beta trend


## Rolling Statistics Challenge: Exponentially Weighted Average 

An alternative way to calculate a rolling window is to take the exponentially weighted moving average. This is like a moving window average, but it assigns greater importance to more recent observations. Try calculating the [`ewm`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.ewm.html) with a 21-day half life for each portfolio, using standard deviation (`std`) as the metric of interest.

In [63]:
# Use `ewm` to calculate the rolling window


---

# Sharpe Ratios
In reality, investment managers and thier institutional investors look at the ratio of return-to-risk, and not just returns alone. After all, if you could invest in one of two portfolios, and each offered the same 10% return, yet one offered lower risk, you'd take that one, right?

### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [64]:
# Annualized Sharpe Ratios


In [65]:
# Visualize the sharpe ratios as a bar plot


### Determine whether the algorithmic strategies outperform both the market (S&P 500) and the whales portfolios.

Write your answer here!

---

# Create Custom Portfolio

In this section, you will build your own portfolio of stocks, calculate the returns, and compare the results to the Whale Portfolios and the S&P 500. 

1. Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.
2. Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock
3. Join your portfolio returns to the DataFrame that contains all of the portfolio returns
4. Re-run the performance and risk analysis with your portfolio to see how it compares to the others
5. Include correlation analysis to determine which stocks (if any) are correlated

## Choose 3-5 custom stocks with at last 1 year's worth of historic prices and create a DataFrame of the closing prices and dates for each stock.

In [66]:
# Reading data from 1st stock


In [67]:
# Reading data from 2nd stock


In [68]:
# Reading data from 3rd stock


In [69]:
# Combine all stocks in a single DataFrame


In [70]:
# Reset Date index


In [71]:
# Reorganize portfolio data by having a column per symbol


In [72]:
# Calculate daily returns

# Drop NAs

# Display sample data


## Calculate the weighted returns for the portfolio assuming an equal number of shares for each stock

In [73]:
# Set weights
weights = [1/3, 1/3, 1/3]

# Calculate portfolio return

# Display sample data


## Join your portfolio returns to the DataFrame that contains all of the portfolio returns

In [74]:
# Join your returns DataFrame to the original returns DataFrame


In [75]:
# Only compare dates where return data exists for all the stocks (drop NaNs)


## Re-run the risk analysis with your portfolio to see how it compares to the others

### Calculate the Annualized Standard Deviation

In [76]:
# Calculate the annualized `std`


### Calculate and plot rolling `std` with 21-day window

In [77]:
# Calculate rolling standard deviation

# Plot rolling standard deviation


### Calculate and plot the correlation

In [78]:
# Calculate and plot the correlation


### Calculate and Plot Rolling 60-day Beta for Your Portfolio compared to the S&P 500

In [79]:
# Calculate and plot Beta


### Using the daily returns, calculate and visualize the Sharpe ratios using a bar plot

In [80]:
# Calculate Annualized Sharpe Ratios


In [81]:
# Visualize the sharpe ratios as a bar plot


### How does your portfolio do?

Write your answer here!